# Notebook Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Constants

In [2]:
TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

# Load the Data

In [3]:
# Features
X_test = np.loadtxt(TEST_FEATURE_MATRIX, delimiter=' ')
# Targets
y_test = np.loadtxt(TEST_TARGET_FILE, delimiter=' ')
# Token probabilities
prob_token_spam = np.loadtxt(TOKEN_SPAM_PROB_FILE, delimiter=' ')
prob_token_ham = np.loadtxt(TOKEN_HAM_PROB_FILE, delimiter=' ')
prob_all_tokens = np.loadtxt(TOKEN_ALL_PROB_FILE, delimiter=' ')

# Claculating the Joint Probability

### The Dot Product

In [4]:
a = np.array([1, 2, 3])
b = np.array([0, 5, 4])
a.dot(b)

22

In [5]:
c = np.array([[0, 6], [3, 0], [5, 1]])
c

array([[0, 6],
       [3, 0],
       [5, 1]])

In [6]:
print(a.dot(c))

[21  9]


Figure out the dimensions of the dot product between X_test and prob_token_spam.

1722 x 1

In [7]:
X_test.shape

(1722, 2500)

In [8]:
prob_token_spam.shape

(2500,)

In [9]:
X_test.dot(prob_token_spam).shape

(1722,)

## Set the Prior

$$P(Spam \, | \, X) = \frac{P(X \, | Spam) \, P(Spam)} {P(X)}$$

In [12]:
PROB_SPAM = 0.3116

Take the log number of a mumpy array. Calculate the log probabilities of the toklens given that the e-mail was spam. This was stored in prob_token_spam.

In [11]:
np.log(prob_token_spam)

array([ -4.40234939,  -5.2663143 ,  -5.0124804 , ..., -11.44943173,
       -12.14257891,  -9.30936557])

# Joint probability in log format

In [13]:
joint_log_spam = X_test.dot(np.log(prob_token_spam) - np.log(prob_all_tokens)) + np.log(PROB_SPAM)

In [14]:
joint_log_spam[:5]

array([12.47901696, 18.88596049,  1.70836529, 18.32541844, 13.80928268])

Calculate the log probability that the e-mails are non-spam given their tokens. Store the result in a variable called joint_log_ham.

$$P(Ham \, | \, X) = \frac{P(X \, | Ham) \, (1-P(Spam))} {P(X)}$$

In [17]:
joint_log_ham = X_test.dot(np.log(prob_token_ham) - np.log(prob_all_tokens)) + np.log(1-PROB_SPAM)

In [18]:
joint_log_ham[:5]

array([-43.576946  , -43.89391298, -32.90817712, -49.50761869,
       -41.80920404])